# Remove obvious outlier matches using GMS

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from DLC_for_WBFM.utils.projects.finished_project_data import ProjectData
import napari
import pandas as pd
import numpy as np
from DLC_for_WBFM.utils.feature_detection.class_frame_pair import FramePair
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

import matplotlib as mpl
mpl.rcParams['font.size'] = 32
from tqdm.auto import tqdm
from DLC_for_WBFM.utils.postures.centerline_pca import WormFullVideoPosture, WormReferencePosture
from DLC_for_WBFM.utils.pipeline.long_range_matching import initialize_worm_object


Bad key "text.kerning_factor" on line 4 in
/home/charles/anaconda3/envs/torch/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# fname = "/scratch/zimmer/Charles/dlc_stacks/worm3-tracklet_wiggle/project_config.yaml"
fname = "/home/charles/dlc_stacks/worm3-newseg-2021_11_17/project_config.yaml"

project_data = ProjectData.load_final_project_data_from_config(fname, to_load_tracklets=True, to_load_frames=True)

Output files: /home/charles/dlc_stacks/worm3-newseg-2021_11_17/3-tracking/manual_annotation/global2tracklet_manual.pickle, /home/charles/dlc_stacks/worm3-newseg-2021_11_17/3-tracking/manual_annotation/manually_split_tracklets.h5, /home/charles/dlc_stacks/worm3-newseg-2021_11_17/3-tracking/manual_annotation/split_names.pickle, /home/charles/dlc_stacks/worm3-newseg-2021_11_17/3-tracking/manual_annotation/split_times.pickle
Project data for directory:
/home/charles/dlc_stacks/worm3-newseg-2021_11_17 
Found the following raw data files:
red_data: True
green_data: True
============Segmentation===============
raw_segmentation: True
segmentation: True
============Tracklets==================
df_training_tracklets: False
reindexed_masks_training: True
============Traces=====================
red_traces: True
green_traces: True
final_tracks: True
behavior_annotations: True



In [3]:
df_tracklets = project_data.df_all_tracklets
segmentation_metadata = project_data.segmentation_metadata
all_frames = project_data.raw_frames
all_matches = project_data.raw_matches
raw_clust = project_data.raw_clust

In [4]:
# fname = "/project/neurobiology/zimmer/wbfm/centerline/wbfm_ulises_centerline_for_charlie/2021-03-04_16-17-30_worm3_ZIM2051-_spline_K.csv"
# fname_X = "/project/neurobiology/zimmer/wbfm/centerline/wbfm_ulises_centerline_for_charlie/2021-03-04_16-17-30_worm3_ZIM2051-_spline_X_coords.csv"
# fname_Y = "/project/neurobiology/zimmer/wbfm/centerline/wbfm_ulises_centerline_for_charlie/2021-03-04_16-17-30_worm3_ZIM2051-_spline_Y_coords.csv"

# full_posture = WormFullVideoPosture(fname, fname_X, fname_Y)
# reference_posture = WormReferencePosture(0, full_posture)

# # Initialize TrackedNeurons at 0, and initialize the TrackedWorm
# # Get all tracklets that start at t=0
# all_tracklet_names = df_tracklets.columns.get_level_values(0).drop_duplicates()

# worm_obj = initialize_worm_object(all_tracklet_names, df_tracklets, raw_clust, segmentation_metadata)

# Visualize a match with outliers

In [6]:
pair = (1,2)

project_data.napari_of_single_match(pair, rigidly_align_volumetric_images=True)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(15.5, 349.5, 449.5), zoom=1.8524320457796852, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=3, last_used=3, range=((0.0, 1.0, 1.0), (0.0, 31.0, 1.0), (0.0, 699.0, 1.0), (0.0, 899.0, 1.0)), current_step=(0, 0, 0, 0), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'raw_red_data' at 0x7f14aa201390>, <Points layer 'n0_zxy' at 0x7f14ab749750>, <Points layer 'n1_zxy' at 0x7f14aa1245d0>, <Tracks layer 'final_matches' at 0x7f14a9f8d210>, <Points layer 'Raw IDs' at 0x7f14aa1a3790>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(visible=Fa

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/site-packages/zmq/eventloop/zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/site-packages/jupyter_client/threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/asyncio/base_events.py", line 563, in run_until_complete
    self._check_runnung()
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/asyncio/base_events.py", line 526, in _check_runnung
    'Cannot run the event loop while another loop is running')
RuntimeError: Cannot run the event loop while another loop is running
ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/charles/anaconda3/envs/torch/lib/pyth

In [18]:
for f in all_frames.values():
    f.video_fname = project_data.project_config.config['preprocessed_red']

# Get a match with outliers

In [26]:
frame_pair_options = all_matches[(0, 1)].options
frame_pair_options.preprocess_using_global_rotation = True
frame_pair_options.add_fdnc_to_candidates = True
this_pair = (1119, 1175)

In [27]:
frame0 = all_frames[this_pair[0]]
frame1 = all_frames[this_pair[1]]

match_to_check = calc_FramePair_from_Frames(frame0, frame1, frame_pair_options=frame_pair_options)

  0%|          | 0/32 [00:00<?, ?it/s]

/home/charles/Current_work/repos/fDNC_Neuron_ID/fDNC/src/model.py:604: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  sents_var = torch.tensor(sents_padded, dtype=torch.float, device=self.device)


In [28]:
v = project_data.napari_of_single_match(this_pair, rigidly_align_volumetric_images=True, this_match=match_to_check);

In [25]:
match_to_check.print_candidates_for_neuron(35)

Candidate: (35, 28, 0.16584736794799584)
Candidate: (35, 32, 0.06513595432206461)


In [29]:
match_to_check.print_candidates_for_neuron(62)

Candidate: (62, 57, 0.1206121522934768)
Candidate: (62, 63, 0.88446075)


# Use GMS to postprocess final matches (and remove obviously wrong ones)

In [30]:
vec, pts0, pts1 = match_to_check.vector_field

In [39]:
dict_pair_to_conf = match_to_check.get_pair_to_conf_dict()

In [31]:
import cv2

In [47]:
# Convert to opencv keypoints and match class
sz = 31.0
kp0 = [cv2.KeyPoint(x, y, 31.0) for z, x, y in pts0]
kp1 = [cv2.KeyPoint(x, y, 31.0) for z, x, y in pts1]

cv2_matches = [cv2.DMatch(int(m0), int(m1), 1.0/conf) for (m0, m1), conf in dict_pair_to_conf.items()]

im1_shape = match_to_check.dat0.shape[1:]
im2_shape = im1_shape

In [60]:
options = {'keypoints1': kp0,
           'keypoints2': kp1,
           'matches1to2': cv2_matches,
           'withRotation': False,
           'thresholdFactor': 1.0}
final_matches = cv2.xfeatures2d.matchGMS(im1_shape, im2_shape, **options)

In [61]:
from DLC_for_WBFM.utils.feature_detection.utils_features import extract_map1to2_from_matches

In [62]:
gms_matches = extract_map1to2_from_matches(final_matches)

In [63]:
initial_matches = match_to_check.get_f0_to_f1_dict()

In [64]:
gms_matches_list = [[k, v, 1.0] for k, v in gms_matches.items()]

In [65]:
match_to_check.final_matches = gms_matches_list

In [67]:
v = project_data.napari_of_single_match(this_pair, rigidly_align_volumetric_images=True, this_match=match_to_check);